In [80]:
#Peak fitting for CdSe
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pybaselines as pybase
import csv

In [81]:
#Import Wavelengths as np array
with open(r"PLWavelengths.csv") as UV_WL:
    wavelengths = np.loadtxt(UV_WL, delimiter=",")
    electronvolts = 1239.84193/wavelengths

In [82]:
#Import PL Intensity data
from numpy import genfromtxt
data = genfromtxt(r"CdSe PL DoE (raw csv).csv", delimiter=',')
PL_data = np.nan_to_num(data, nan=0, posinf=0)

In [ ]:
#Code for fitting PL Peaks
from pylab import *
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as mpl
import pybaselines as pybase

def findPL(y,wavelengths):

    #clean data to remove nan and inf
    m = np.stack((wavelengths,y),axis=0)
    m = np.transpose(m)
    m = (m[~np.isnan(m).any(axis=1), :])
    m = (m[~np.isinf(m).any(axis=1), :])
    wavelengths = m[:,0]
    y = m[:,1]
    #Smooth curves with Whittaker Eilers
    [y_filt,_] = pybase.whittaker.aspls(y, lam=500.0, diff_order=2, max_iter=100, tol=0.001, weights=None, alpha=None)
    [y_base,_] = pybase.whittaker.aspls(y, lam=5000.0, diff_order=2, max_iter=100, tol=0.001, weights=None, alpha=None)
    #Subtract the minimum
    min_val = np.ndarray.min(y_base[650:])
    max_val = np.ndarray.max(y_base[650:])
    y = (y - min_val)*(1/max_val)
    y_filt = (y_filt - min_val)*(1/max_val)
    #Take derivative and smooth with Whittaker Eilers
    dydx = np.diff(y_filt)/np.diff(wavelengths)
    [dydx_filt,_] = pybase.whittaker.aspls(dydx, lam=10000.0, diff_order=2, max_iter=100, tol=0.001, weights=None, alpha=None)
    #Take derivative and smooth with Whittaker Eilers (double derivative)
    d2ydx2 = np.diff(dydx_filt)/np.diff(wavelengths)[1:]
    [d2ydx2_filt,_] = pybase.whittaker.aspls(d2ydx2, lam=5000.0, diff_order=2, max_iter=100, tol=0.001, weights=None, alpha=None)

    from scipy.signal import find_peaks, peak_prominences

    peak_idx, _ = find_peaks(-d2ydx2_filt[628:])
    prominences = 10000*peak_prominences(-d2ydx2_filt[628:], peak_idx)[0]
    a = np.stack((prominences, peak_idx, wavelengths[630:][peak_idx],y_filt[630:][peak_idx]),axis = 0)
    a = np.transpose(a)
    a = np.delete(a, np.where(a[:,-1] <= 0.01)[0], axis=0)
    sorted_peaks = a[a[:, 0].argsort()]
    peak_est = sorted_peaks[-1,2]
    height_est = sorted_peaks[-1,3]
    peak_index_est = sorted_peaks[-1,1]
    peak_idx2, _ = find_peaks(d2ydx2_filt[630:])
    
    print(peak_est)


    right = np.argmax(peak_idx2>peak_index_est)
    right_idx = peak_idx2[right]+40
    left_idx = peak_idx2[right-1]-40
    left_height = y_filt[630:][left_idx]
    right_height = y_filt[630:][right_idx]

    x = wavelengths[630:]
    y = y_filt[630:]
    def gauss(x,mu,sigma,A):
        return A*exp(-(x-mu)**2/2/sigma**2)

    def bimodal(x,mu1,sigma1,A1,mu2,sigma2,A2):
        return gauss(x,mu1,sigma1,A1)+gauss(x,mu2,sigma2,A2)

    print((height_est-right_height)/(height_est-left_height))

    if (height_est-right_height)/(height_est-left_height) < 0.99 or (height_est-right_height)/(height_est-left_height) >1.01 :

        expected=(peak_est,16,height_est,650,85,0.1)
        params,cov=curve_fit(bimodal,x,y,expected,maxfev=5000)
        sigma=sqrt(diag(cov))
        fig = plt.figure(figsize=(12, 8), dpi=80)
        plt.plot(x,bimodal(x,*params),color='red',lw=3,label='model')
        plt.plot(x,y)
        legend()
        params = abs(np.reshape(params, (2, 3)))  
        print(params)
        sorted_params = params[params[:, 1].argsort()]

        if abs(params[0,0]-params[1,0])>30:
            print(sorted_params)  
            y = y_filt[630:]-gauss(x,sorted_params[-1,0],sorted_params[-1,1],sorted_params[-1,2])
            plt.plot(x,y)
        if abs(params[0,0]-params[1,0])<=30:
            y = y_filt[630:]
            plt.plot(x,y)


    if (height_est-right_height)/(height_est-left_height) > 0.99 and (height_est-right_height)/(height_est-left_height) <1.01 :
        y = y_filt[630:]
        plt.plot(x,y)


    peak_idx, _ = find_peaks(y)
    prominences = peak_prominences(y, peak_idx)[0]
    a = np.stack((prominences, peak_idx, x[peak_idx],y[peak_idx]),axis = 0)
    a = np.transpose(a)
    sorted_peaks = a[a[:, 0].argsort()]
    peak_est = sorted_peaks[-1,2]
    height_est = sorted_peaks[-1,3]
    print(sorted_peaks[-1,:])

    max_peak_idx = sorted_peaks[-1,1]
    max_peak_idx = max_peak_idx.astype(int)
    peak = x[max_peak_idx]
    print(peak)
    peak_height = y[max_peak_idx]
    half_height_idx_left = np.argmax(y[0:max_peak_idx]>peak_height/2)
    print(x[half_height_idx_left])
    half_height_idx_right = np.argmax(y[max_peak_idx:]<peak_height/2)
    print(x[max_peak_idx+half_height_idx_right])
    fwhm = x[max_peak_idx+half_height_idx_right]-x[half_height_idx_left]
    peak_ev = 1239.84193/(x[max_peak_idx])
    fwhm_ev = 1239.84193/x[half_height_idx_left] - 1239.84193/x[max_peak_idx+half_height_idx_right]
    peak_height = peak_height*max_val
  
    return([peak,fwhm,peak_height,peak_ev,fwhm_ev,peak_height])
    return(fig)

In [ ]:
#Run peak fit code on dataset
np.set_printoptions(edgeitems=8, precision=3, suppress=True, threshold=5)
parameters = np.zeros((np.shape(PL_data)[0],13))
index = 0
experiment_no = 1
for row in PL_data:
    print(index+1)
    parameters[index,0:7] = PL_data[index,0:7]
    parameters[index,7:13] = findPL(PL_data[index,7:],wavelengths)
    print(parameters[index,7:13])
    plt.show()
    index += 1

In [133]:
#Save raw data
np.savetxt("CdSe PL fitting (DoE raw).csv", parameters, delimiter=",",fmt='%1.3f')

In [139]:
#Import raw data
from numpy import genfromtxt
data = genfromtxt(r"CdSe PL fitting (DoE raw).csv", delimiter=',')
Fit_data = abs(np.nan_to_num(data, nan=0, posinf=0))

In [140]:
#Clean raw data
index = 0
for row in Fit_data:
    if Fit_data[index,8] >= 60 or Fit_data[index,8] <= 18: #check for feasibility of fwhm
        Fit_data[index,:] = np.zeros((1,13))
    elif Fit_data[index,7] < 400 or Fit_data[index,7] > 620: #Check feasibility of wavelength
        Fit_data[index,:] = np.zeros((1,13))
    elif Fit_data[index,9] < 300: #Check feasibility of height
        Fit_data[index,:] = np.zeros((1,13))
    index += 1
Clean_fit = Fit_data[~np.all(Fit_data == 0, axis=1)]

In [141]:
#Save cleaned data
np.savetxt("CdSe PL fitting (DoE cleaned).csv", Clean_fit, delimiter=",",fmt='%1.3f')